# Input Statistical Analysis for all datasets

In [30]:
import numpy as np
import pandas as pd
import spacy
import emoji
from collections import Counter

In [4]:
DATA_DIR = '../data'

### Utility Functions

In [73]:
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

def slice_dataframe_and_compute_word_frequency(df, slice_cols, slice_vals, text_col, spacy_lang_pkg):
    sliced_df = df.copy()
    for i in range(len(slice_cols)):
        sliced_df = sliced_df[sliced_df[slice_cols[i]] == slice_vals[i]]
    print(f'Found a total of {len(sliced_df)} examples')
    nlp = spacy.load(spacy_lang_pkg)
    text = ' '.join(sliced_df[text_col])
    text = emoji.get_emoji_regexp().sub(r'', text)
    doc = nlp(text)
    words = [token.text for token in doc if not token.is_stop and not token.is_punct and len(token) > 1]
    freqs = Counter(words)
    return freqs

### Spanish (Basile et al.)

In [74]:
df = pd.read_csv(f'{DATA_DIR}/spanish-basile/hateval2019_es_train.csv')

In [75]:
df.head()

,id,text,HS,TR,AG
0,20001,Easyjet quiere duplicar el número de mujeres p...,1,0,0
1,20002,El gobierno debe crear un control estricto de ...,1,0,0
2,20003,Yo veo a mujeres destruidas por acoso laboral ...,0,0,0
3,20004,"— Yo soy respetuoso con los demás, sólamente l...",0,0,0
4,20007,Antonio Caballero y como ser de mal gusto e ig...,0,0,0


#### 1.1 Word Frequecy for Examples that are Hate Speech

In [80]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['HS'], [1], 'text', 'es_core_news_sm')
freqs.most_common()[:10]

Found a total of 1857 examples


[('puta', 418),
 ('perra', 345),
 ('zorra', 216),
 ('Cállate', 122),
 ('callate', 110),
 ('inmigrantes', 102),
 ('mujer', 96),
 ('Callate', 91),
 ('mereces', 82),
 ('cállate', 78)]

#### 1.2 Word Frequecy for Examples that are Hate Speech and Aggressive

In [81]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['HS', 'AG'], [1, 1], 'text', 'es_core_news_sm')
freqs.most_common()[:10]

Found a total of 1502 examples


[('puta', 398),
 ('perra', 322),
 ('zorra', 207),
 ('Cállate', 122),
 ('callate', 109),
 ('Callate', 90),
 ('mereces', 80),
 ('cállate', 78),
 ('mierda', 74),
 ('PUTA', 67)]

#### 1.3 Word Frequecy for Examples that are Hate Speech but not Aggressive

In [82]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['HS', 'AG'], [1, 0], 'text', 'es_core_news_sm')
freqs.most_common()[:10]

Found a total of 355 examples


[('mujer', 54),
 ('inmigrantes', 36),
 ('mujeres', 32),
 ('subsaharianos', 23),
 ('perra', 23),
 ('puta', 20),
 ('España', 17),
 ('país', 17),
 ('papeles', 16),
 ('inmigrante', 15)]

#### 2.  Word Frequecy for Examples that are not Hate Speech

In [83]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['HS'], [0], 'text', 'es_core_news_sm')
freqs.most_common()[:10]

Found a total of 2643 examples


[('puta', 784),
 ('hijo', 237),
 ('acoso', 198),
 ('madre', 149),
 ('perra', 142),
 ('polla', 141),
 ('PUTA', 139),
 ('mierda', 117),
 ('mereces', 112),
 ('violación', 110)]